In [1]:
!pip install underthesea
!pip install demoji
!pip install pyvi

In [2]:
!apt update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (138 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
spark-3.3.0-bin-hadoop3/
spark-3.3.0-bin-hadoop3

In [3]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer, VectorIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import LogisticRegression, LinearSVC
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import regex
from pyspark.mllib.evaluation import MulticlassMetrics
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql import types
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
from pyspark.ml.clustering import *
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import rand, col, isnan, when, count, udf, lit, length

from pyspark.ml.feature import Tokenizer, RegexTokenizer,StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import os

In [6]:
import findspark
findspark.init()

In [7]:
spark = SparkSession.builder.appName('Sentiment_Analysis').getOrCreate()

In [8]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [9]:
%cd '/content/gdrive/MyDrive/Colab/LDS0_K294_T2C7/PROJECT3/'

/content/gdrive/MyDrive/Colab/LDS0_K294_T2C7/PROJECT3


In [10]:
data = spark.read.csv('/content/gdrive/MyDrive/Colab/LDS0_K294_T2C7/PROJECT3/data/pre_data.csv', header=True, inferSchema=True).drop('_c0')

In [11]:
# df['target'] = pd.cut(df['Rating'], bins=[0,4.5,7.5,10], labels=[0, 1, 2])

In [12]:
data.show()

+--------------------+------+-------+--------------------+
|             Comment|Rating|  class|         Comment_pre|
+--------------------+------+-------+--------------------+
|     Đã thư rất ngon|   5.0|Neutral|                 thư|
|Tệ mua mang về ch...|   1.0|Dislike|tệ gói tương_ớt t...|
|21h30 2 đứa nhỏ k...|   4.2|Dislike|kêu đói hồi bình_...|
|Nay có khuyến mại...|   7.4|   Like|khuyến_mại tặng t...|
|Thề lâu lắm rồi t...|   1.0|Dislike|thề lắm tức_giận ...|
|Nếu đây là lần đầ...|  10.0|   Like|đầu thử vị thần_t...|
|Đặt giao combo C ...|   1.0|Dislike|add phô_tính tiền...|
|Hic phần cơm Mc D...|   5.6|Neutral|hích cơm dở ăn_tâ...|
|Mình tới đây vào ...|   4.2|Dislike|tối_ngày thái_độ ...|
|Sáng nay 19 5 202...|   1.0|Dislike|ghé đakao hướng d...|
|Giao thiếu gà và ...|   3.4|Dislike|    gà hàng bảo kiểm|
|Mình đi với đứa b...|   7.8|   Like|đi tại_chỗ cá tặn...|
|Ăn tại chỗ không ...|   1.0|Dislike|tại_chỗ không_đặt...|
|Càng ngày chất lư...|   6.8|Neutral|chất_lượng chán b..

In [13]:
# Create the 'Sentiment' column based on the 'Rating' values
data = data.withColumn(
    'Sentiment',
    when(col('Rating') <= 4.5, 0).  # Condition for Rating <= 4.5
    when((col('Rating') > 4.5) & (col('Rating') <= 7.4), 1).  # Condition for 4.6 <= Rating <= 7.4
    otherwise(2)  # Condition for Rating >= 7.5
)

# Show the resulting DataFrame
data.show()

+--------------------+------+-------+--------------------+---------+
|             Comment|Rating|  class|         Comment_pre|Sentiment|
+--------------------+------+-------+--------------------+---------+
|     Đã thư rất ngon|   5.0|Neutral|                 thư|        1|
|Tệ mua mang về ch...|   1.0|Dislike|tệ gói tương_ớt t...|        0|
|21h30 2 đứa nhỏ k...|   4.2|Dislike|kêu đói hồi bình_...|        0|
|Nay có khuyến mại...|   7.4|   Like|khuyến_mại tặng t...|        1|
|Thề lâu lắm rồi t...|   1.0|Dislike|thề lắm tức_giận ...|        0|
|Nếu đây là lần đầ...|  10.0|   Like|đầu thử vị thần_t...|        2|
|Đặt giao combo C ...|   1.0|Dislike|add phô_tính tiền...|        0|
|Hic phần cơm Mc D...|   5.6|Neutral|hích cơm dở ăn_tâ...|        1|
|Mình tới đây vào ...|   4.2|Dislike|tối_ngày thái_độ ...|        0|
|Sáng nay 19 5 202...|   1.0|Dislike|ghé đakao hướng d...|        0|
|Giao thiếu gà và ...|   3.4|Dislike|    gà hàng bảo kiểm|        0|
|Mình đi với đứa b...|   7.8|   Li

In [14]:
# Select only the required columns
data = data.select('Rating', 'Comment_pre', 'Sentiment')

# Show the resulting DataFrame
data.show()

+------+--------------------+---------+
|Rating|         Comment_pre|Sentiment|
+------+--------------------+---------+
|   5.0|                 thư|        1|
|   1.0|tệ gói tương_ớt t...|        0|
|   4.2|kêu đói hồi bình_...|        0|
|   7.4|khuyến_mại tặng t...|        1|
|   1.0|thề lắm tức_giận ...|        0|
|  10.0|đầu thử vị thần_t...|        2|
|   1.0|add phô_tính tiền...|        0|
|   5.6|hích cơm dở ăn_tâ...|        1|
|   4.2|tối_ngày thái_độ ...|        0|
|   1.0|ghé đakao hướng d...|        0|
|   3.4|    gà hàng bảo kiểm|        0|
|   7.8|đi tại_chỗ cá tặn...|        2|
|   1.0|tại_chỗ không_đặt...|        0|
|   6.8|chất_lượng chán b...|        1|
|   1.0|miếng ứng nuốt_kh...|        0|
|   1.0|drive through kiể...|        0|
|   8.0|thương_hiệu fastf...|        2|
|   1.0|  gà không_nhân_viên|        0|
|   1.0|gà không_cho bé g...|        0|
|   1.2|chú_ý ít_ức ức gà...|        0|
+------+--------------------+---------+
only showing top 20 rows



In [15]:
data = data.withColumnRenamed('Comment_pre','text').withColumnRenamed('Sentiment','label')
data.show(5)

+------+--------------------+-----+
|Rating|                text|label|
+------+--------------------+-----+
|   5.0|                 thư|    1|
|   1.0|tệ gói tương_ớt t...|    0|
|   4.2|kêu đói hồi bình_...|    0|
|   7.4|khuyến_mại tặng t...|    1|
|   1.0|thề lắm tức_giận ...|    0|
+------+--------------------+-----+
only showing top 5 rows



# Data Analysis

In [16]:
# Count the number of rows
data.count()

26579

In [17]:
# Check NaN
is_nan = data.select([count(when(isnan(c), c)).alias(c) for c in data.columns])
is_nan.show()

+------+----+-----+
|Rating|text|label|
+------+----+-----+
|     0|   0|    0|
+------+----+-----+



In [18]:
# Check Null
is_null = data.select([count(when(col(c).contains('None') | col(c).contains('null') | (col(c) == '') | col(c).isNull(), c)).alias(c) for c in data.columns])
is_null.show()

+------+----+-----+
|Rating|text|label|
+------+----+-----+
|     0|   0|    0|
+------+----+-----+



In [19]:
# Check duplicates
data.count() - data.distinct().count()

1149

In [20]:
# Drop duplicates
data = data.drop_duplicates()

In [21]:
# After drop
data.count()

25430

In [22]:
data = data.select(['text', 'label'])
data.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|    gà hàng bảo kiểm|    0|
|hôm xế_chiều buồn...|    2|
|gà giòn cành hông...|    1|
|dịp ghé quán đườn...|    2|
|người_yêu trưa ho...|    2|
+--------------------+-----+
only showing top 5 rows



# Clean and Prepare the Data

In [23]:
data = data.withColumn('length',length(data['text']))

In [24]:
data.show(5)

+--------------------+-----+------+
|                text|label|length|
+--------------------+-----+------+
|    gà hàng bảo kiểm|    0|    16|
|hôm xế_chiều buồn...|    2|   192|
|gà giòn cành hông...|    1|    52|
|dịp ghé quán đườn...|    2|    83|
|người_yêu trưa ho...|    2|   109|
+--------------------+-----+------+
only showing top 5 rows



In [25]:
# Pretty Clear Difference
data.groupby('label').mean().show()

+-----+----------+------------------+
|label|avg(label)|       avg(length)|
+-----+----------+------------------+
|    1|       1.0| 141.0425942753294|
|    2|       2.0|131.32481438061743|
|    0|       0.0|123.08744453145393|
+-----+----------+------------------+



# Feature Transformations : Tokenizer, StopWordsRemover, CountVectorizer, IDF

In [26]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

In [27]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

# Building model

In [28]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier


nb   = NaiveBayes(featuresCol='features', labelCol='label', predictionCol='prediction')
tree = DecisionTreeClassifier(featuresCol='features', labelCol='label', predictionCol='prediction')
rfc  = RandomForestClassifier(featuresCol='features', labelCol='label', predictionCol='prediction')
gbt  = GBTClassifier(featuresCol='features', labelCol='label', predictionCol='prediction')

# Pipeline

In [29]:
data_prep_pipe = Pipeline(stages=[
                              tokenizer,
                              stopremove,
                              count_vec,
                              idf,
                              clean_up])

In [30]:
cleaner = data_prep_pipe.fit(data)

In [31]:
clean_data = cleaner.transform(data)
clean_data.show(5)

+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|label|length|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    gà hàng bảo kiểm|    0|    16|[gà, hàng, bảo, k...|[gà, hàng, bảo, k...|(43765,[2,23,179,...|(43765,[2,23,179,...|(43766,[2,23,179,...|
|hôm xế_chiều buồn...|    2|   192|[hôm, xế_chiều, b...|[hôm, xế_chiều, b...|(43765,[0,2,12,17...|(43765,[0,2,12,17...|(43766,[0,2,12,17...|
|gà giòn cành hông...|    1|    52|[gà, giòn, cành, ...|[gà, giòn, cành, ...|(43765,[2,30,33,4...|(43765,[2,30,33,4...|(43766,[2,30,33,4...|
|dịp ghé quán đườn...|    2|    83|[dịp, ghé, quán, ...|[dịp, ghé, quán, ...|(43765,[0,2,11,14...|(43765,[0,2,11,14...|(43766,[0,2,11,14...|
|người_yêu tr

# Training and Evaluation!

In [32]:
clean_data = clean_data.select(['label', 'features'])
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(43766,[2,23,179,...|
|    2|(43766,[0,2,12,17...|
|    1|(43766,[2,30,33,4...|
|    2|(43766,[0,2,11,14...|
|    2|(43766,[0,2,3,13,...|
+-----+--------------------+
only showing top 5 rows



In [33]:
training, testing = clean_data.randomSplit([0.80, 0.20])

In [34]:
# Chỉ thay đổi thuật toán chổ này nếu áp dụng nhiều thuật toán
# Khi dùng nhiều model thì khai báo một lượt các model , fit -> transform -> evaluate một lượt --> So sánh kết quả --> kết luận
nb_model = nb.fit(training)
tree_model = tree.fit(training)
rfc_model = rfc.fit(training)
#gbt_model = gbt.fit(training)

In [35]:
data.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: integer (nullable = false)
 |-- length: integer (nullable = true)



In [36]:
nb_test_result   = nb_model.transform(testing)
tree_test_result = tree_model.transform(testing)
rfc_test_result  = rfc_model.transform(testing)
#gbt_test_result  = gbt_model.transform(testing)

In [37]:
nb_test_result.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(43766,[0,1,2,4,6...|[-2848.2973409802...|[6.3357310766053E...|       1.0|
|    0|(43766,[0,1,2,4,7...|[-2103.6575357972...|[1.0,2.7218698514...|       0.0|
|    0|(43766,[0,1,2,5,9...|[-2345.6295033411...|[0.99999999999998...|       0.0|
|    0|(43766,[0,1,2,10,...|[-1373.2390831258...|[1.0,8.8078352014...|       0.0|
|    0|(43766,[0,1,3,8,2...|[-1757.7892817982...|[0.99875642962054...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [38]:
tree_test_result.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(43766,[0,1,2,4,6...|   [279.0,32.0,13.0]|[0.86111111111111...|       0.0|
|    0|(43766,[0,1,2,4,7...|  [26.0,137.0,800.0]|[0.02699896157840...|       2.0|
|    0|(43766,[0,1,2,5,9...|   [279.0,32.0,13.0]|[0.86111111111111...|       0.0|
|    0|(43766,[0,1,2,10,...|[2189.0,6546.0,93...|[0.12098601669153...|       2.0|
|    0|(43766,[0,1,3,8,2...|     [52.0,30.0,7.0]|[0.58426966292134...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [39]:
rfc_test_result.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(43766,[0,1,2,4,6...|[2.82141931790422...|[0.14107096589521...|       2.0|
|    0|(43766,[0,1,2,4,7...|[4.01253574400956...|[0.20062678720047...|       2.0|
|    0|(43766,[0,1,2,5,9...|[4.97866850719064...|[0.24893342535953...|       2.0|
|    0|(43766,[0,1,2,10,...|[3.80537699434363...|[0.19026884971718...|       2.0|
|    0|(43766,[0,1,3,8,2...|[3.51486011460530...|[0.17574300573026...|       2.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [40]:
#gbt_test_result.show(5)

In [41]:
# confusion matrix
nb_test_result.groupBy("label", "prediction").count().show()
tree_test_result.groupBy("label", "prediction").count().show()
rfc_test_result.groupBy("label", "prediction").count().show()
#gbt_test_result.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    2|       1.0|  714|
|    1|       0.0|  277|
|    0|       2.0|   32|
|    0|       0.0|  608|
|    1|       2.0|  662|
|    2|       2.0| 1661|
|    1|       1.0|  836|
|    2|       0.0|  131|
|    0|       1.0|  151|
+-----+----------+-----+

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    2|       1.0|    2|
|    1|       0.0|   77|
|    0|       2.0|  585|
|    0|       0.0|  198|
|    1|       2.0| 1698|
|    2|       2.0| 2483|
|    2|       0.0|   21|
|    0|       1.0|    8|
+-----+----------+-----+

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       2.0|  791|
|    1|       2.0| 1774|
|    2|       2.0| 2506|
|    1|       1.0|    1|
+-----+----------+-----+



# MulticlassClassificationEvaluator

In [42]:
nb_predictions  = nb_model.transform(testing)
dtc_predictions = tree_model.transform(testing)
rfc_predictions = rfc_model.transform(testing)
#gbt_predictions = gbt_model.transform(testing)

In [43]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol='label',
                                                  predictionCol='prediction',
                                                  metricName='accuracy')

In [44]:
nb_acc  = acc_evaluator.evaluate(nb_predictions)
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
#gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [45]:
print("Results")

print('-'*60)
print('A NaiveBayes has an accuracy of: {0:2.2f}%'.format(nb_acc*100))
print('-'*60)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*60)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
#print('-'*60)
#print('A gradient-boosted tree has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Results
------------------------------------------------------------
A NaiveBayes has an accuracy of: 61.22%
------------------------------------------------------------
A single decision tree has an accuracy of: 52.86%
------------------------------------------------------------
A random forest ensemble has an accuracy of: 49.43%
